# Regime Exploration Notebook

Interactive exploration of market regimes using multivariate HMM.

This notebook helps you:
1. Load historical data for your target asset
2. Train multivariate HMM with regime detection
3. Visualize regime transitions
4. Analyze regime characteristics (returns, volatility, etc.)
5. Design trading rules based on regime observations

In [ ]:
# Setup
import os
os.chdir('/workspace')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Hidden Regime
import hidden_regime as hr

# Plotting
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

print('Imports successful!')

## Step 1: Load Data

In [ ]:
# Configuration
TICKER = 'SPY'  # Change this to your target asset
START_DATE = '2015-01-01'
END_DATE = '2024-01-01'
N_STATES = 3  # Number of regimes

print(f'Loading {TICKER} data...')
print(f'Period: {START_DATE} to {END_DATE}')
print(f'Regime states: {N_STATES}')

# Create pipeline
pipeline = hr.create_financial_pipeline(
    ticker=TICKER,
    n_states=N_STATES,
    start_date=START_DATE,
    end_date=END_DATE,
    include_report=False
)

print('\nRunning regime detection...')
result = pipeline.update()

print(f'\nData loaded: {len(result)} observations')
print(f'Date range: {result.index[0].date()} to {result.index[-1].date()}')

result.head()

## Step 2: Visualize Regime Transitions

In [ ]:
# Plot price and regimes
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Price
ax1.plot(result.index, result['close'], label='Close Price', color='black', linewidth=2)
ax1.set_ylabel('Price')
ax1.set_title(f'{TICKER} Price History')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Regime with color coding
regime_colors = {'Bull': 'green', 'Bear': 'red', 'Sideways': 'gray', 'Crisis': 'darkred'}
for regime, color in regime_colors.items():
    mask = result['regime_name'] == regime
    ax2.scatter(result[mask].index, result[mask]['close'], label=regime, color=color, s=10, alpha=0.6)
ax2.set_ylabel('Price')
ax2.set_title('Regime Classifications')
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

# Confidence
ax3.fill_between(result.index, 0, result['confidence'], alpha=0.3, color='blue')
ax3.plot(result.index, result['confidence'], label='Confidence', color='blue', linewidth=2)
ax3.axhline(0.65, color='orange', linestyle='--', label='Threshold (65%)')
ax3.set_ylabel('Confidence')
ax3.set_xlabel('Date')
ax3.set_title('Regime Detection Confidence')
ax3.set_ylim([0, 1])
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f'Total observations: {len(result)}')

## Step 3: Regime Statistics

In [ ]:
# Calculate statistics by regime
regime_stats = []

for regime in result['regime_name'].unique():
    regime_data = result[result['regime_name'] == regime]
    
    if len(regime_data) == 0:
        continue
    
    # Returns
    returns = regime_data['log_return'].dropna()
    
    stats = {
        'Regime': regime,
        'Days': len(regime_data),
        'Avg Daily Return': returns.mean() * 100,
        'Volatility': returns.std() * 100,
        'Total Return': (np.exp(returns.sum()) - 1) * 100,
        'Win Days': (returns > 0).sum(),
        'Win Rate': (returns > 0).sum() / len(returns) * 100,
        'Sharpe': returns.mean() / returns.std() * np.sqrt(252) if returns.std() > 0 else 0,
        'Max DD': regime_data['drawdown'].min() * 100 if 'drawdown' in regime_data else -returns.max() * 100,
    }
    
    regime_stats.append(stats)

stats_df = pd.DataFrame(regime_stats)

print('\n' + '='*80)
print('REGIME STATISTICS')
print('='*80)
print(stats_df.to_string(index=False))
print()

# Regime transitions
transitions = result['regime_name'].diff().fillna(result['regime_name'].iloc[0]) != result['regime_name']
n_transitions = transitions.sum() - 1  # -1 to exclude first row

print(f'Total regime changes: {n_transitions}')
print(f'Average regime duration: {len(result) / (n_transitions + 1):.0f} days')
print()

## Step 4: Performance by Regime

In [ ]:
# Plot cumulative returns by regime
fig, ax = plt.subplots(figsize=(14, 6))

regime_colors = {'Bull': 'green', 'Bear': 'red', 'Sideways': 'gray', 'Crisis': 'darkred'}

for regime in result['regime_name'].unique():
    if pd.isna(regime):
        continue
    
    regime_data = result[result['regime_name'] == regime].copy()
    regime_data['cumulative_return'] = (1 + regime_data['log_return']).cumprod()
    
    ax.plot(
        regime_data.index,
        regime_data['cumulative_return'],
        label=regime,
        linewidth=2,
        color=regime_colors.get(regime, 'black')
    )

ax.set_ylabel('Cumulative Return')
ax.set_xlabel('Date')
ax.set_title(f'{TICKER}: Cumulative Returns by Regime')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Step 5: Regime Characteristics (Volatility, Returns)

In [ ]:
# Box plots for returns and volatility by regime
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Returns by regime
regime_list = []
returns_list = []

for regime in result['regime_name'].unique():
    if pd.isna(regime):
        continue
    regime_data = result[result['regime_name'] == regime]
    returns = regime_data['log_return'].dropna() * 100
    
    regime_list.extend([regime] * len(returns))
    returns_list.extend(returns)

box_data = pd.DataFrame({'Regime': regime_list, 'Daily Return (%)': returns_list})
sns.boxplot(data=box_data, x='Regime', y='Daily Return (%)', ax=ax1, palette='Set2')
ax1.set_title('Daily Returns Distribution by Regime')
ax1.axhline(0, color='red', linestyle='--', alpha=0.5)
ax1.grid(True, alpha=0.3, axis='y')

# Volatility by regime  
vol_list = []
vol_regime_list = []

for regime in result['regime_name'].unique():
    if pd.isna(regime):
        continue
    regime_data = result[result['regime_name'] == regime]
    # Rolling volatility
    rolling_vol = regime_data['log_return'].rolling(20).std() * 100 * np.sqrt(252)
    rolling_vol = rolling_vol.dropna()
    
    vol_list.extend(rolling_vol)
    vol_regime_list.extend([regime] * len(rolling_vol))

vol_data = pd.DataFrame({'Regime': vol_regime_list, 'Realized Vol (%)': vol_list})
sns.boxplot(data=vol_data, x='Regime', y='Realized Vol (%)', ax=ax2, palette='Set2')
ax2.set_title('Realized Volatility by Regime')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Step 6: Design Trading Rules

In [ ]:
# Based on observations, design allocation rules
print('\n' + '='*80)
print('SUGGESTED ALLOCATION RULES (Customize based on your analysis above)')
print('='*80)

# Find best performing regime
best_regime = stats_df.loc[stats_df['Total Return'].idxmax(), 'Regime']
worst_regime = stats_df.loc[stats_df['Total Return'].idxmin(), 'Regime']

print(f'\nBest performing regime: {best_regime} ({stats_df.loc[stats_df["Regime"] == best_regime, "Total Return"].values[0]:.1f}%)')
print(f'Worst performing regime: {worst_regime} ({stats_df.loc[stats_df["Regime"] == worst_regime, "Total Return"].values[0]:.1f}%)')

print(f'\nExample Allocation Rules:')
print(f'  Bull regime (high confidence):   100% {TICKER}')
print(f'  Sideways regime (70% conf):      50% {TICKER}, 50% Bonds')
print(f'  Bear regime (high confidence):   100% Bonds/Gold')
print(f'  Low confidence (<65%):           Balanced allocation')

print(f'\nNext Step: Create your strategy using generate_strategy.py')
print(f'  python scripts/generate_strategy.py --name my_strategy --template custom')

## Step 7: Export for Strategy Development

In [ ]:
# Save analysis results for reference
analysis_results = {
    'ticker': TICKER,
    'n_states': N_STATES,
    'start_date': START_DATE,
    'end_date': END_DATE,
    'regime_stats': stats_df.to_dict('records'),
    'n_transitions': int(n_transitions),
    'best_regime': best_regime,
    'worst_regime': worst_regime,
}

# Save to JSON
import json
with open(f'regime_analysis_{TICKER}_{datetime.now().strftime("%Y%m%d")}.json', 'w') as f:
    json.dump(analysis_results, f, indent=2, default=str)

print(f'Analysis saved to regime_analysis_{TICKER}_{datetime.now().strftime("%Y%m%d")}.json')
print('\nReady to create your strategy!')